Where to get training data: http://help.sentiment140.com/for-students

In [66]:
import pandas as pd

import tweet_proccess_funcs
from tweet_proccess_funcs import process_tweet
from gensim.models import word2vec
from nltk import word_tokenize

In [84]:
columns = ['polarity', 'tweet_id', 'date', 'query', 'twitter_handle', 'tweet']
training = pd.read_csv('processed_train.csv',encoding='latin-1',index_col=0)

In [85]:
training = training.sample(frac=.1)
print(len(training))
training.head()

160000


,polarity,tweet_id,date,query,twitter_handle,tweet,processed_tweets
1008584,4,1880812384,Fri May 22 02:35:36 PDT 2009,NO_QUERY,ceno_byte,@Ashtarte Ahh. I'll sort out the router this...,"['ashtarte', 'ahh', 'ill', 'sort', 'router', '..."
1434081,4,2060563625,Sat Jun 06 19:08:29 PDT 2009,NO_QUERY,leahhitchens,@davidkillimayer I would like you to know I wi...,"['davidkillimayer', 'would', 'like', 'know', '..."
538718,0,2199191924,Tue Jun 16 17:08:56 PDT 2009,NO_QUERY,misfit_narciss,Were all just broken hearts,"['broken', 'heart']"
1181836,4,1982150701,Sun May 31 10:37:46 PDT 2009,NO_QUERY,__zues,"Junkyard was fun, Steve wore my glasses, http:...","['junkyard', 'fun', 'steve', 'wore', 'glass', ..."
18612,0,1556643647,Sat Apr 18 23:41:41 PDT 2009,NO_QUERY,mistersaxon,@JohnKremer and 85% of statistics are made up ...,"['johnkremer', 'statistic', 'made', 'spot', 'h..."


In [90]:
training = training.drop('processed_tweets',axis=1)

In [92]:
training['processed_tweets'] = training.tweet.apply(process_tweet)

In [69]:
training['processed_tweets'] = training.processed_tweets.apply(lambda x: x[1:-1].split(','))

In [93]:
training['processed_tweets'].head()

1008584    [ashtarte, ahh, ill, sort, router, weekend, like]
1434081    [davidkillimayer, would, like, know, opining, ...
538718                                       [broken, heart]
1181836    [junkyard, fun, steve, wore, glass, httptinycc...
18612      [johnkremer, statistic, made, spot, httpisgdrl...
Name: processed_tweets, dtype: object

In [94]:
target = training['polarity']
data = training['processed_tweets'].values

In [96]:
all_words_list = [item for sublist in training.processed_tweets for item in sublist]

In [103]:
total_vocab = set(all_words_list)

In [104]:
print(len(all_words_list))
len(unique_words)

1238957


149926

In [106]:
import numpy as np
glove = {}
with open('glove.6B.50d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocab:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [107]:
glove['lol']

array([-0.54289 ,  0.053743, -0.46978 ,  0.17652 , -0.33742 ,  0.16731 ,
        1.0061  ,  0.23131 , -0.99158 ,  0.98234 , -0.33188 ,  0.64598 ,
        0.071642, -0.28759 , -0.30952 , -0.96751 ,  0.22189 ,  0.84231 ,
        0.55649 , -0.50558 , -0.28354 ,  0.5827  ,  1.7962  ,  0.44665 ,
        0.046912,  0.99483 ,  0.086517,  0.75013 ,  0.11787 , -0.68936 ,
       -0.91459 ,  0.36019 , -0.68144 ,  0.55527 , -1.0641  , -0.29523 ,
        0.033354, -1.111   ,  0.71948 ,  0.66059 ,  0.89162 ,  0.14977 ,
       -1.4292  ,  0.067031,  0.12727 , -0.21811 , -0.51361 ,  0.20745 ,
       -0.074958,  0.080575], dtype=float32)

In [108]:
class W2vVectorizer(object):
    
    def __init__(self, w2v):
        # Takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
    
    # Note: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # it can't be used in a scikit-learn pipeline  
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

In [109]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf =  Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
              ('Random Forest', RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
              ('Logistic Regression', LogisticRegression())])

In [110]:
models = [('Random Forest', rf),
          ('Support Vector Machine', svc),
          ('Logistic Regression', lr)]

In [111]:
scores = [(name, cross_val_score(model, data, target, cv=2).mean()) for name, model, in models]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.4s finished
/Users/Allie/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for al

In [112]:
scores

[('Random Forest', 0.67275),
 ('Support Vector Machine', 0.69471875),
 ('Logistic Regression', 0.66435)]